In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
url = "https://www.popvortex.com/music/charts/top-100-songs.php"

In [3]:
# download html with a get request
response=requests.get(url)
response.status_code

200

In [4]:
# parse html (create the 'soup')
soup = BeautifulSoup(response.content, "html.parser")

In [5]:
#soup

In [6]:
#chart-position-1 > div.chart-content.col-xs-12.col-sm-8 > p

In [7]:
#soup.select("div.chart-content.col-xs-12.col-sm-8 > p")

In [8]:
#chart-position-1 > div.chart-content.col-xs-12.col-sm-8 > p > cite

In [9]:
#figured out that if cite is for titles and em for artists

song_artist_pairs = soup.find_all('p', class_='title-artist')


songs = [pair.find('cite').text for pair in song_artist_pairs]
artists = [pair.find('em').text for pair in song_artist_pairs]

In [10]:
df = pd.DataFrame({
        'Song Title': songs,
        'Artist': artists
    })

In [11]:
df

,Song Title,Artist
0,Lovin On Me,Jack Harlow
1,Lil Boo Thang,Paul Russell
2,White Horse,Chris Stapleton
3,I Remember Everything (feat. Kacey Musgraves),Zach Bryan
4,Need A Favor,Jelly Roll
...,...,...
95,Autumn Comes Dry,Luminara Stellaris
96,Last Christmas (Single Version),Wham!
97,In My Life,The Beatles
98,TRUCK BED,HARDY


In [12]:
url_billboard="https://www.billboard.com/charts/hot-100/"

In [13]:
# download html with a get request
response=requests.get(url_billboard)
response.status_code

200

In [14]:
# parse html (create the 'soup')
soup_billboard = BeautifulSoup(response.content, "html.parser")

In [15]:
song_elements = soup_billboard.select("h3.c-title")

In [24]:
#song_elements

In [17]:
artist_elements = soup_billboard.select("#post-1479786 > div.pmc-paywall > div > div > div > div.chart-results-list.\/\/.lrv-u-padding-t-150.lrv-u-padding-t-050\@mobile-max > div > ul > li.lrv-u-width-100p > ul > li.o-chart-results-list__item.\/\/.lrv-u-flex-grow-1.lrv-u-flex.lrv-u-flex-direction-column.lrv-u-justify-content-center.lrv-u-border-b-1.u-border-b-0\@mobile-max.lrv-u-border-color-grey-light.lrv-u-padding-l-050.lrv-u-padding-l-1\@mobile-max > span")

In [18]:
# so in here is better to get a higher level of the data, not too deep to get all the info and then from there get the title and the artist

data = [] #make a dictionary that will have the data 

for e in soup_billboard.find_all(attrs={'class':'o-chart-results-list-row-container'}): # so this is the container for all the info we need
    data.append({
        'Song Title':e.h3.get_text(strip=True),
        'Artist':e.h3.find_next('span').get_text(strip=True)
    })

data_frame_billboard = pd.DataFrame(data)

In [19]:
data_frame_billboard

,Song Title,Artist
0,Cruel Summer,Taylor Swift
1,Paint The Town Red,Doja Cat
2,Is It Over Now? (Taylor's Version) [From The V...,Taylor Swift
3,Snooze,SZA
4,Standing Next To You,Jung Kook
...,...,...
95,Clean (Taylor's Version),Taylor Swift
96,Bleed,The Kid LAROI
97,But I Got A Beer In My Hand,Luke Bryan
98,We Don't Fight Anymore,Carly Pearce Featuring Chris Stapleton


In [20]:
hot_songs=pd.concat([data_frame_billboard,df],axis=0)
hot_songs

,Song Title,Artist
0,Cruel Summer,Taylor Swift
1,Paint The Town Red,Doja Cat
2,Is It Over Now? (Taylor's Version) [From The V...,Taylor Swift
3,Snooze,SZA
4,Standing Next To You,Jung Kook
...,...,...
95,Autumn Comes Dry,Luminara Stellaris
96,Last Christmas (Single Version),Wham!
97,In My Life,The Beatles
98,TRUCK BED,HARDY


In [21]:
hot_songs.duplicated().sum()

17

In [22]:
hot_songs=hot_songs.drop_duplicates()

In [23]:
hot_songs.duplicated().sum()

0